In [ ]:
%reload_ext setup

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
# Cell 2: Read and combine prompt files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Read common and supplier matching prompts
general_context = read_file('../prompts/general_context.txt')
matching_rules = read_file('../prompts/matching_rules.txt')
distribution_rules = read_file('../prompts/distribution_rules.txt')
matcher_prompt = read_file('../prompts/matcher/matcher_1.txt')
critic_prompt = read_file('../prompts/critic/critic_1.txt')

# Read subsidy matching prompts
subsidy_matching_rules = read_file('../prompts/subsidy_matching_rules.txt')
pricing_rules = read_file('../prompts/pricing_rules.txt')
subsidy_matcher_prompt = read_file('../prompts/subsidy_matcher/subsidy_matcher_1.txt')
subsidy_critic_prompt = read_file('../prompts/subsidy_critic/subsidy_critic_1.txt')

# Combine prompts
combined_matcher_prompt = matcher_prompt.format(general_context=general_context, matching_rules=matching_rules, distribution_rules=distribution_rules)
combined_critic_prompt = critic_prompt.format(general_context=general_context, matching_rules=matching_rules, distribution_rules=distribution_rules)

combined_subsidy_matcher_prompt = subsidy_matcher_prompt.format(general_context=general_context, subsidy_matching_rules=subsidy_matching_rules, pricing_rules=pricing_rules)
combined_subsidy_critic_prompt = subsidy_critic_prompt.format(general_context=general_context, subsidy_matching_rules=subsidy_matching_rules, pricing_rules=pricing_rules)

## Demo (~100 registrants)


In [ ]:
# Cell 3: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt=combined_matcher_prompt,
    critic_prompt=combined_critic_prompt
)

# Cell 3: Run the chat with matching task
message = (
    "Load data from '../data/demo/registration.json' and '../data/demo/offers.json'. "
    "Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), "
    "and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. "
    "Save match JSON result to '../data/demo/matches.json'."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Read and combine prompt files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

general_context = read_file('../prompts/general_context.txt')
subsidy_matching_rules = read_file('../prompts/subsidy_matching_rules.txt')
pricing_rules = read_file('../prompts/pricing_rules.txt')
subsidy_matcher_prompt = read_file('../prompts/subsidy_matcher/subsidy_matcher_1.txt')
subsidy_critic_prompt = read_file('../prompts/subsidy_critic/subsidy_critic_1.txt')

combined_subsidy_matcher_prompt = subsidy_matcher_prompt.format(general_context=general_context, subsidy_matching_rules=subsidy_matching_rules, pricing_rules=pricing_rules)
combined_subsidy_critic_prompt = subsidy_critic_prompt.format(general_context=general_context, subsidy_matching_rules=subsidy_matching_rules, pricing_rules=pricing_rules)


# Cell 3: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt=combined_subsidy_matcher_prompt,
    critic_prompt=combined_subsidy_critic_prompt
)

# Cell 3: Run the chat with matching task
message = (
    "Offers: ```json"
    f"{read_file('../data/demo/offers.json')}```"
    "Registrations: ```json"
    f"{read_file('../data/demo/registration.json')}```"
    "Incentives: ```json"   
    f"{read_file('../data/demo/incentives.json')}```"
    "Tthe most importantly, match: ```json"
    f"{read_file('../data/demo/matches.json')}```"
    "Use instructions in the <subsidy_matching_rules> and <pricing_rules>."
    "Save incentives matches in JSON format to '../data/demo/pos.json'."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

## Full run

In [ ]:

# Cell 3: Function to save JSON data
def save_json(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Data saved to {file_path}")

# Cell 4: Step 1 - Match registrations to suppliers
async def match_registrations_to_suppliers():
    print("Step 1: Matching registrations to suppliers...")
    
    # Get the agent for supplier matching
    supplier_agents = await get_agents(
        state_path="model_config.yaml",
        matcher_prompt=combined_matcher_prompt,
        critic_prompt=combined_critic_prompt
    )
    
    # Run the chat with matching task
    message = (
        "Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. "
        "Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), "
        "and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. "
        "Save match JSON result to '../data/demo/matches.json'."
    )
    
    matches_data = None
    
    async for msg in supplier_agents.run_stream(
        task=[TextMessage(content=message, source="user")],
        cancellation_token=CancellationToken()
    ):
        if isinstance(msg, TextMessage):
            print(f"{msg.source}: {msg.content}")
            
            # Try to extract JSON data from the message
            if msg.source == "matcher" and matches_data is None:
                try:
                    # Look for JSON content in the message
                    start_idx = msg.content.find('{')
                    end_idx = msg.content.rfind('}')
                    if start_idx != -1 and end_idx != -1:
                        json_str = msg.content[start_idx:end_idx+1]
                        matches_data = json.loads(json_str)
                except json.JSONDecodeError:
                    pass
                
        elif isinstance(msg, TaskResult):
            final_message = "Task completed. "
            if msg.stop_reason:
                final_message += f"Stop reason: {msg.stop_reason}"
            print(final_message)
    
    # Save matches to file
    if matches_data:
        save_json(matches_data, )
    else:
        print("Warning: No valid JSON data was extracted from the matcher's response.")
    
    return matches_data

# Cell 5: Step 2 - Match to incentives
async def match_to_incentives():
    print("\nStep 2: Matching to incentives...")
    
    # Get the agent for subsidy matching
    subsidy_agents = await get_agents(
        state_path="model_config.yaml",
        matcher_prompt=combined_subsidy_matcher_prompt,
        critic_prompt=combined_subsidy_critic_prompt
    )
    
    # Run the chat with subsidy matching task
    message = (
        "Load data from '../data/demo/matches.json', '../data/demo/offers.json' and '../data/demo/incentives.json'. "
        "Match all outputs to incentives. "
        "Output matches in JSON format as per the matching rules."
    )
    
    pos_data = None
    
    async for msg in subsidy_agents.run_stream(
        task=[TextMessage(content=message, source="user")],
        cancellation_token=CancellationToken()
    ):
        if isinstance(msg, TextMessage):
            print(f"{msg.source}: {msg.content}")
            
            # Try to extract JSON data from the message
            if msg.source == "matcher" and pos_data is None:
                try:
                    # Look for JSON content in the message
                    start_idx = msg.content.find('{')
                    end_idx = msg.content.rfind('}')
                    if start_idx != -1 and end_idx != -1:
                        json_str = msg.content[start_idx:end_idx+1]
                        pos_data = json.loads(json_str)
                except json.JSONDecodeError:
                    pass
                
        elif isinstance(msg, TaskResult):
            final_message = "Task completed. "
            if msg.stop_reason:
                final_message += f"Stop reason: {msg.stop_reason}"
            print(final_message)
    
    # Save POS data to file
    if pos_data:
        save_json(pos_data, '../data/demo/pos.json')
    else:
        print("Warning: No valid JSON data was extracted from the subsidy matcher's response.")
    
    return pos_data

# Cell 6: Run the complete workflow
async def run_complete_workflow():
    # Step 1: Match registrations to suppliers
    await match_registrations_to_suppliers()
    
    # Step 2: Match to incentives
    await match_to_incentives()
    
    print("\nComplete workflow finished!")

# Execute the workflow
await run_complete_workflow()